In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
full_df = []
directory = "SP_Experiência/2018/"
for filename in os.listdir(directory):
    print(filename)
    df = pd.read_csv(directory+filename,sep = ';',encoding="ISO-8859-1")[['DATA_VENDA','CATEGORIA_1','CATEGORIA_2','CATEGORIA_3','CPFCNPJ']]
    df.CPFCNPJ = df.CPFCNPJ.astype(str)
    df["CAT_COMB"] = df.CATEGORIA_1 + df.CATEGORIA_2 + df.CATEGORIA_3
    df = df.drop(columns=['CATEGORIA_1','CATEGORIA_2','CATEGORIA_3'])
    df = df[df.CPFCNPJ != 'nan']
    full_df.append(df)
df = pd.concat(full_df)
del(full_df)
df['item'] = df.CAT_COMB.astype("category").cat.codes
df = df.sort_values(by=['item'])
items = list(set(df.CAT_COMB.to_list()))
cats = list(set(df.item.to_list()))
del(df)
df = pd.DataFrame(list(zip(items, cats)), 
               columns =['CAT_COMB', 'id']) 

Venda Detalhada_211018_311018.txt


/home/guilherme/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Venda Detalhada_210818_310818.txt
Venda Detalhada_010818_100818.txt
Venda Detalhada_010918_100918.txt
Venda Detalhada_110918_200918.txt
Venda Detalhada_210918_300918.txt
Venda Detalhada_111018_201018.txt
Venda Detalhada_211218_311218.txt
Venda Detalhada_111218_201218.txt
Venda Detalhada_110818_200818.txt
Venda Detalhada_211118_301118.txt
Venda Detalhada_011018_101018.txt
Venda Detalhada_011218_101218.txt
Venda Detalhada_011118_101118.txt
Venda Detalhada_111118_201118.txt


In [3]:
#partidos.index = pd.to_datetime(partidos['created_at'])
#partidos = partidos.groupby([partidos.index.year, partidos.index.month,partidos.index.day, partidos.partido])
df.index = pd.to_datetime(df.DATA_VENDA, format="%Y-%m-%d-%H.%M.%S.%f")
df = df.groupby(['CPFCNPJ',df.index.year, df.index.month, df.index.day])

In [4]:
def cria_itemsets(serie):
    compras_dia = []
    for item in serie:
        compras_dia.append(item)
    return sorted(list(set(compras_dia)))

df = df.agg({'item': cria_itemsets})
df.index.names = ['CPFCNPJ', 'ano', 'mes', 'dia']
df = df.reset_index()
df = df[df.CPFCNPJ != "nan"]

In [5]:
def cria_sequencias(serie):
    seq_individuo = []
    for item in serie:
        seq_individuo.append(item)
    return seq_individuo
df = df.groupby(['CPFCNPJ'])
df = df.agg({'item': cria_sequencias})

In [6]:
df.columns

Index(['item'], dtype='object')

In [7]:
df

,item
CPFCNPJ,
1000017680.0,"[[300, 301, 306, 321, 530]]"
10000242888.0,"[[47, 50, 51, 175, 186, 189, 300, 306, 310, 32..."
1000029280.0,"[[319, 380, 445, 501, 722]]"
1000030584.0,"[[300, 301, 302, 306, 310], [300, 301, 302, 30..."
10000334812.0,"[[300, 301, 302, 303, 306, 310, 321]]"
...,...
99998823820.0,[[501]]
9999928560.0,"[[292, 302, 305, 306, 310, 414, 435]]"
99999323120.0,"[[90, 96, 105, 128, 140, 147, 153, 282, 283, 2..."


In [8]:
df['seq_len'] = df.item.apply(len)

In [9]:
df.sort_values(by = ['seq_len'], ascending = False)

,item,seq_len
CPFCNPJ,,
38030169000248.0,"[[4, 8, 9, 16, 17, 19, 82, 87, 89, 90, 91, 93,...",151
38030169000590.0,"[[4, 6, 9, 12, 14, 16, 17, 18, 19, 21, 82, 85,...",151
38030169000914.0,"[[4, 14, 16, 17, 18, 19, 21, 74, 81, 82, 87, 8...",151
38030169001139.0,"[[4, 6, 9, 16, 17, 19, 21, 79, 82, 85, 87, 89,...",151
5951362001072.0,"[[722], [722], [722], [722], [403, 442, 722], ...",147
...,...,...
5259586220.0,[[307]],1
27596567649.0,"[[182, 208]]",1
27596434843.0,"[[51, 56, 301, 303, 306, 310, 311, 319, 321, 3...",1


In [10]:
df = df[df.seq_len > 3]
df

,item,seq_len
CPFCNPJ,,
10000242888.0,"[[47, 50, 51, 175, 186, 189, 300, 306, 310, 32...",8
1000030584.0,"[[300, 301, 302, 306, 310], [300, 301, 302, 30...",4
10000359726.0,"[[66, 300, 302, 306, 310, 319, 321, 422, 501, ...",21
10000527882.0,"[[4, 298, 309, 313, 632], [283, 307], [307, 30...",5
1000098681.0,"[[221, 306, 387, 617], [306], [200, 306], [306...",7
...,...,...
9999288480.0,"[[300, 306, 310, 311, 321, 323], [306, 310, 31...",16
9999674076.0,"[[175, 200, 239, 246, 294, 435], [239, 246, 30...",10
9999746084.0,"[[310, 401, 407, 442], [178], [196, 223, 310, ...",10


In [11]:
df.to_csv("trajetorias_2018.csv")